In [2]:
import pandas as pd
import numpy as np
import csv

In [61]:
filter_names = ['gold', 'dji', 'mcsf', 'oil', 'shanghai', 'sp500']

In [84]:
filtered_dates = set()
for name in filter_names:
    filtered_df = pd.read_csv(f'..\Res\{name}.csv')
    if name not in ['gold', 'shanghai', 'sp500']:
        filtered_df = filtered_df.loc[(filtered_df.Volume > -0.0001)]
    dates = filtered_df["Date"]
    if len(filtered_dates) == 0:
        filtered_dates = set([date.replace('-', '').strip() for date in dates])
        continue
    filtered_dates = filtered_dates.intersection(set([date.replace('-', '').strip() for date in dates]))

In [86]:
len(filtered_dates)

5755

In [6]:
all_events_counts = pd.read_csv('..\Res\yearly_country.csv',
                               names=['Date', 'Country', 'EventsCount'],)

In [7]:
processed_countries = ['CN', 'FR', 'DE', 'GB', 'US']
dates = [i for i in range(1995, 2020)]

In [8]:
all_events_counts = all_events_counts[all_events_counts['Date'].isin(dates)]
all_events_counts = all_events_counts[all_events_counts['Country'].isin(processed_countries)]

In [9]:
events_count = all_events_counts.groupby(['Date']).sum().reset_index()

In [10]:
events_count.at[24, 'EventsCount'] = events_count.at[24, 'EventsCount'] * (12/10)

In [11]:
min_event = events_count['EventsCount'].min()
events_count['Normalizations'] = min_event / events_count['EventsCount']
del events_count['EventsCount']

In [12]:
events_count

,Date,Normalizations
0,1995,1.000000
1,1996,0.700116
2,1997,0.631965
3,1998,0.546790
4,1999,0.514594
5,2000,0.535283
6,2001,0.390997
7,2002,0.458946
8,2003,0.351956
9,2004,0.440101


In [26]:
def year(date):
    return date[:4]

def month(date):
    return date[:6]

def isUSA(country):
    return 1.05 if 'USA' == country else 1.

def isUS(country):
    return 1.05 if 'US' == country else 1.

def group_by_day(x):
    d = {}
    d['AllNumMentions'] = x['AllNumMentions'].sum() * x['Normalizations'].mean()
    d['Goldstein'] = x['goldsteinscale'].mean()
    d['MonthYear'] = x['monthyear'].mean()
    return pd.Series(d)

def group_by_day_no_norm(x):
    d = {}
    d['AllNumMentions'] = x['AllNumMentions'].sum()
    d['Goldstein'] = x['Goldstein'].mean()
    d['MonthYear'] = x['MonthYear'].mean()
    return pd.Series(d)

def to_base(cameo_code):
    if cameo_code.startswith('0'):
        cameo_code = cameo_code[1]
    else:
        cameo_code = cameo_code[:2]
    return cameo_code

sign = lambda x: (1, -1)[x < 0]

In [13]:
from sklearn import linear_model

In [36]:
def mention_regression(mentions):
    regr = linear_model.LinearRegression()
    x = np.arange(0, len(mentions), 1)
    regr.fit(x.reshape(-1, 1), mentions)
    mentions_pred = regr.predict(x.reshape(-1,1))
    return [abs(mention) / mention_pred for mention, mention_pred in zip(mentions, mentions_pred)]

In [19]:
full_df = pd.DataFrame()
for part in range(1, 27):
    df = pd.read_csv(f'..\Res\exports_all_part{part}.csv', 
                  sep='\t', 
                  names=['globaleventid', 'day', 'monthyear', 'actor1name', 'actor1countrycode', 'actor2name', 'actor2countrycode',
                        'eventcode', 'eventbasecode',
                        'eventrootcode', 'goldsteinscale', 'nummentions', 'numsources', 'avgtone', 'actor1geo_fullname',
                        'actor1geo_countrycode', 'actor2geo_fullname', 'actor2geo_countrycode', 'actiongeo_fullname',
                        'actiongeo_countrycode'],
                dtype={'globaleventid':str, 'day':str, 'monthyear':str, 'actor1name':str, 'actor1countrycode':str, 'actor2name':str, 'actor2countrycode':str,
                        'eventcode':str, 'eventbasecode':str,
                        'eventrootcode':str, 'goldsteinscale':float, 'nummentions':int, 'numsources':int, 'avgtone':float, 'actor1geo_fullname':str,
                        'actor1geo_countrycode':str, 'actor2geo_fullname':str, 'actor2geo_countrycode':str, 'actiongeo_fullname':str,
                        'actiongeo_countrycode':str})
    del df['globaleventid']
    del df['actor1name']
    del df['actor2name']
    del df['eventbasecode']
    del df['actor1geo_fullname']
    del df['actor2geo_fullname']
    del df['actiongeo_fullname']
    del df['actor2geo_countrycode']
    del df['actor1geo_countrycode']
    del df['avgtone']
    
    df = df[df['day'].isin(filtered_dates)]

    df['year'] = df['day'].apply(year).astype('int')
    df['monthyear'] = df['monthyear'].astype('int')
    df['day'] = df['day'].astype('int')

    df['AllNumMentions'] = (df['numsources'] + df['nummentions']) * df['actor1countrycode'].apply(isUSA) * df['actor2countrycode'].apply(isUSA) * df['actiongeo_countrycode'].apply(isUS)
    
    del df['actor1countrycode']
    del df['actor2countrycode']
    del df['actiongeo_countrycode']
    del df['eventrootcode']
    
    df = pd.merge(df, events_count, left_on='year', right_on='Date', how='left')
    df.drop(['Date'], axis=1, inplace=True)
    
    df = df.dropna()
    
    df = df.groupby(['day', 'eventcode']).apply(group_by_day).reset_index()
    
    full_df = full_df.append(df)
    print(f'{part}. iteration ended.')
    part += 1 

1. iteration ended.
2. iteration ended.
3. iteration ended.
4. iteration ended.
5. iteration ended.
6. iteration ended.
7. iteration ended.
8. iteration ended.
9. iteration ended.
10. iteration ended.
11. iteration ended.
12. iteration ended.
13. iteration ended.
14. iteration ended.
15. iteration ended.
16. iteration ended.
17. iteration ended.
18. iteration ended.
19. iteration ended.
20. iteration ended.
21. iteration ended.
22. iteration ended.
23. iteration ended.
24. iteration ended.
25. iteration ended.
26. iteration ended.


In [21]:
full_df.head()

,day,eventcode,AllNumMentions,Goldstein,MonthYear
0,19950102,010,241.080,0.0,199501.0
1,19950102,012,79.660,-0.4,199501.0
2,19950102,013,53.700,0.4,199501.0
3,19950102,014,39.555,0.0,199501.0
4,19950102,015,15.605,0.0,199501.0


In [27]:
full_df = full_df.groupby(['day', 'eventcode']).apply(group_by_day_no_norm).reset_index()

In [28]:
full_df['BaseCode'] = full_df['eventcode'].apply(to_base)

In [29]:
events_codes = full_df['eventcode'].unique()

In [37]:
scaled_df = pd.DataFrame()
for idx, code in enumerate(events_codes):
    df = full_df[full_df['eventcode'] == code]
    df['ScaledMentions'] = mention_regression(list(df['AllNumMentions']))
    scaled_df = scaled_df.append(df)
    print(f'{idx + 1}. iteration ended.')
del full_df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

1. iteration ended.
2. iteration ended.
3. iteration ended.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

4. iteration ended.
5. iteration ended.
6. iteration ended.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

...

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


207. iteration ended.
208. iteration ended.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


209. iteration ended.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [38]:
scaled_df['Coeff'] = scaled_df['AllNumMentions'] * np.power(1.3,abs(scaled_df['Goldstein'])) * scaled_df['Goldstein'].apply(sign)
del scaled_df['eventcode']
del scaled_df['AllNumMentions']
del scaled_df['Goldstein']

In [39]:
scaled_df = scaled_df.groupby(['day', 'BaseCode']).sum().reset_index()

In [87]:
scaled_df = scaled_df[scaled_df['day'].isin(filtered_dates)]

In [40]:
cameo_codes = scaled_df['BaseCode'].unique()
cameo_codes = sorted(list(map(int, cameo_codes)))
labels = ['code_' + str(code) for code in cameo_codes]

In [42]:
def empty_row(): 
    return {label: 0 for label in labels}

In [88]:
import csv

row_to_write = ''
current_date = '19950102'
writer_dict = empty_row()
with open(r'E:\Studia IET\eksploracja\LSTM_Stock_Prediction\Res\events_filtered_scaled.csv', mode='w', encoding="utf8") as csv_filtered_file:
    csv_writer = csv.DictWriter(csv_filtered_file, fieldnames=labels)
    csv_writer.writeheader()
    for row in scaled_df.itertuples():
        date = int(row.day)
        if current_date != row.day:
            csv_writer.writerow(writer_dict)
            writer_dict = empty_row()
            current_date = row.day
        writer_dict['code_' + row.BaseCode] = row.Coeff

In [89]:
len(scaled_df['day'].unique())

5755